# Proyecto MusicStream – Año 2016

Extracción de datos desde Spotify y Last.fm usando funciones separadas.
Lógica clara, modular y fácil de explicar.

## 1. Importaciones y configuración

In [2]:
# IMPORTAR LIBRERIAS
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
 #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np #para convertir formato nan de Paython a none de MySQL
import mysql.connector #para conectar Paython con MySQL
from mysql.connector import Error # para capturar errores de MySQL


load_dotenv() # Carga las variables definidas en el archivo .env // Devuelve True si se cargan correctamente, False si no
host=os.getenv("MYSQL_HOST") # Obtiene el valor de la variable de entorno MYSQL_HOST
user=os.getenv("MYSQL_USER") # Obtiene el usuario de la base de datos
password=os.getenv("MYSQL_PASSWORD") # Obtiene la contraseña de la base de datos

## 2. Credenciales y conexión Spotify

In [3]:
# --- COMPROBACIÓN Y CONEXIÓN CON SPOTIFY ---
# Recupera las credenciales del archivo .env
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Verifica que ambas credenciales estén disponibles
if not client_id or not client_secret:
    print("⚠️ ADVERTENCIA: Las credenciales de Spotify no están cargadas. Revisa tu archivo .env.")
else:
    # Configura la autenticación segura con Spotify usando OAuth de aplicación
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    # Inicializa el cliente de Spotify con las credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    print("✅ Conexión con Spotify inicializada.")

✅ Conexión con Spotify inicializada.


In [4]:
# --- PRUEBA DE CONEXIÓN CON SPOTIFY ---
# Objetivo: comprobar que el cliente de Spotify funciona y devuelve datos válidos.

# Realizamos una búsqueda de artistas por género (genre solo funciona con artistas)
prueba = spotify.search(
    q='genre:latin',   # Buscamos artistas del género "latin"
    type='artist',     # Tipo de búsqueda: artistas
    limit=1            # Solo 1 resultado, suficiente para probar
)

# Extraemos el primer artista de los resultados
artista = prueba['artists']['items'][0]

# Mostramos el nombre del artista para confirmar que la conexión y la búsqueda funcionan
print("✅ Conexión exitosa con Spotify")
print("🎵 Artista de prueba encontrado:", artista['name'])

✅ Conexión exitosa con Spotify
🎵 Artista de prueba encontrado: Bad Bunny


# 3. Extracción de datos Spotify

In [5]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA HACER SEARCH
# ────────────────────────────────────────────────
def spotify_search_seguro(spotify, query, tipo, limite=10, reintentos=3):
    """
    Función que busca en Spotify de forma segura.
    Detecta el error 429 (rate limit) y espera si es necesario.
    
    Parámetros:
    - spotify: objeto de la API
    - query: texto a buscar
    - tipo: "artist" o "album"
    - limite: máximo resultados
    - reintentos: cuántas veces intentar si hay error 429
    """
    intento = 0  # Contador de intentos

    while intento < reintentos:
        try:
            # Llamada a Spotify
            return spotify.search(q=query, type=tipo, limit=limite)

        except SpotifyException as e:
            if e.http_status == 429:
                # Si nos dicen "espera", tomamos el tiempo de Retry-After
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Demasiadas peticiones. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1  # Sumamos intento
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Capturamos cualquier error inesperado
            print("❌ Error inesperado:", e)
            return None

    # Si agotamos todos los intentos
    print("⚠️ Demasiados intentos. Se omite esta búsqueda.")
    return None


In [6]:
def busqueda_spotify(generos, año, spotify):
    todas_las_canciones = []          # Lista para almacenar todas las canciones encontradas
    albumes_ya_vistos = set()         # Conjunto para evitar álbumes duplicados

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # Busca artistas del género usando la función segura
        resultado_artistas = spotify_search_seguro(spotify, query=f"genre:{genero}", tipo="artist", limite=50)
        if resultado_artistas is None:
            continue  # Saltamos si hubo error
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # Busca álbumes del artista en el año indicado
            busqueda = f"artist:{nombre_artista} year:{año}"
            resultado_albumes = spotify_search_seguro(spotify, query=busqueda, tipo="album", limite=50)
            if resultado_albumes is None:
                continue
            albumes = resultado_albumes["albums"]["items"]

            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]

                if id_album in albumes_ya_vistos:
                    continue
                albumes_ya_vistos.add(id_album)

                # Obtiene canciones del álbum
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print("  → Canciones de este género añadidas\n")

    # Resumen final
    print("RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in todas_las_canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")
    print("Total de canciones encontradas:", len(todas_las_canciones))

    return pd.DataFrame(todas_las_canciones)


In [ ]:
# LLAMADA A LA FUNCION PARA EXTRAER DATOS DE SPOTIFY
genero = ["pop","latin","jazz","rock"] # Lista de géneros musicales a analizar
año = 2016 # año seleccionado
canciones_2016_df = busqueda_spotify(genero,año, spotify) # Ejecuta la búsqueda en Spotify y devuelve los resultados en un DataFrame

Buscando canciones del año 2016
Género: pop


Your application has reached a rate/request limit. Retry will occur after: 74911 s


In [ ]:
canciones_2016_df  # Muestra el contenido completo del DataFrame


,nombre,artista,album,genero,año
0,Einaudi: Elegy For The Arctic,Ludovico Einaudi,Elegy For The Arctic,classical,2016
1,Drop - Mogwai Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
2,Drop - Starkey Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
3,Elements - DJ Tennis Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
4,Night - Lone Wolf Trait Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
...,...,...,...,...,...
24549,The Hunter,Red Hot Chili Peppers,The Getaway,rock,2016
24550,Dreams of a Samurai,Red Hot Chili Peppers,The Getaway,rock,2016
24551,Dark Necessities,Red Hot Chili Peppers,Dark Necessities,rock,2016
24552,We Turn Red,Red Hot Chili Peppers,We Turn Red,rock,2016


In [8]:
canciones_2016_df.head() # Muestra las primeras filas del DataFrame para inspeccionar la estructura

,nombre,artista,album,genero,año
0,Einaudi: Elegy For The Arctic,Ludovico Einaudi,Elegy For The Arctic,classical,2016
1,Drop - Mogwai Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
2,Drop - Starkey Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
3,Elements - DJ Tennis Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016
4,Night - Lone Wolf Trait Remix,Ludovico Einaudi,Elements (The Remixes),classical,2016


In [9]:
canciones_2016_df.info() # Muestra información estructural del DataFrame (columnas, tipos y nulos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24554 entries, 0 to 24553
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   nombre   24554 non-null  object
 1   artista  24554 non-null  object
 2   album    24554 non-null  object
 3   genero   24554 non-null  object
 4   año      24554 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 959.3+ KB


In [10]:
canciones_2016_df.isnull().sum() # Cuenta los valores nulos (NaN) por columna

nombre     0
artista    0
album      0
genero     0
año        0
dtype: int64

# 4. Credenciales y conexión LAST.FM

In [11]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
api_key_lastfm = os.getenv("API_KEY_LASTFM")

# Verifica que la clave exista antes de usarla
if not api_key_lastfm:
    print("⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.")
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print("✅ Conexión con Last.fm inicializada.")

✅ Conexión con Last.fm inicializada.


In [12]:
# --- PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM ---
if not api_key_lastfm:
    print("❌ Clave API de Last.fm no definida.")
else:
    try:
        # Hacemos una petición mínima para verificar la conexión usando el método 'chart.getTopArtists'
        respuesta = requests.get(
            "http://ws.audioscrobbler.com/2.0/",
            params={
                'method': 'chart.gettopartists',
                'api_key': api_key_lastfm,
                'format': 'json',
                'limit': 1  # Solo necesitamos 1 artista para probar
            }
        )
        respuesta.raise_for_status()
        print("✅ Conexión exitosa con Last.fm")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al conectar con Last.fm: {e}")

✅ Conexión exitosa con Last.fm


# 5. Extracción de datos LAST.FM

In [13]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/") # URL base de la API de Last.fm para realizar consultas

In [14]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista)                        # Codifica espacios/caracteres especiales para URL
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"                # URL base de la API de Last.fm
    params_info = {                                                   # Parámetros de la consulta
        'method': 'artist.getinfo',                                   # Método API para info del artista
        'artist': artista_codificado,                                 # Nombre del artista codificado
        'api_key': api_key_lastfm,                                     # Clave de API
        'format': 'json'                                              # Formato de respuesta JSON
    }
    
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)  # Llamada GET con timeout
        response.raise_for_status()                                     # Lanza error si HTTP status != 200
        data = response.json()                                          # Convierte respuesta a diccionario
        
        if "artist" in data:                                           # Verifica que se encontró el artista
            artista_info = data['artist']                              
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()  # Extrae resumen sin enlaces
            return {
                'bio_resumen': bio_summary,                             # Resumen de biografía
                'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),  # Oyentes
            }
        else:
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}  # Artista no encontrado
        
    except requests.exceptions.RequestException as e:                   # Captura errores de red/HTTP
        status_code = getattr(e.response, 'status_code', 'N/A')        # Obtiene código de error si existe
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:                                             # Captura errores de procesamiento
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:                                                # Verifica si la API key está configurada
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = canciones_2016_df['artista'].unique()          # Extrae artistas únicos
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")
    
    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista'])  # DataFrame temporal de artistas
    print("\nIniciando consultas a Last.fm...")
    
    resultados_lastfm_serie = artistas_df['artista'].apply(           # Aplica la función a cada artista
        busqueda_info_artista, 
        args=(api_key_lastfm,)                                        # Pasa la API key como argumento adicional
    )
    
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)       # Convierte Serie de diccionarios a DataFrame
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)             # Añade columna 'artista' al inicio
    
    df_final = pd.merge(                                               # Une DataFrames principal y Last.fm
        canciones_2016_df,
        datos_lastfm_df,                                               # DataFrame con datos de Last.fm
        on='artista',                                                  # Clave de unión
        how='left'                                                     # Mantiene todas las canciones aunque falten datos
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")  # Mensaje final



Total de artistas únicos a consultar en Last.fm: 142

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [15]:
datos_lastfm_df # Muestra el contenido completo del DataFrame

,artista,bio_resumen,listeners
0,Ludovico Einaudi,Ludovico Maria Enrico Einaudi OMRI (born 23 No...,1764546
1,Yiruma,Yiruma is a piano music composer from South Ko...,835393
2,Karl Jenkins,Karl Jenkins (b. 17 february 1944) is a Welsh ...,398120
3,Yann Tiersen,"Yann Tiersen (born in Brest, Brittany, France ...",1907976
4,Joe Hisaishi,"Mamoru Fujisawa, known professionally as Joe H...",1156492
...,...,...,...
137,Elton John,Sir Elton Hercules John (born Reginald Kenneth...,5080474
138,Queen,Queen are a British rock band formed in London...,7052203
139,Maná,There is more than one artist with this name:\...,818403
140,Juanes,Juan Esteban Aristizábal Vásquez (born August ...,1171691


In [16]:
datos_lastfm_df.info() # Muestra información estructural del DataFrame (columnas, tipos y nulos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artista      142 non-null    object
 1   bio_resumen  142 non-null    object
 2   listeners    142 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.5+ KB


# CREACION BASE DE DATOS

In [ ]:
# FASE 2: CREACIÓN DE LA BASE DE DATOS (VERSIÓN PYTHON)

# A. Conexión a MySQL
try:
    # Intentamos conectarnos al servidor MySQL
    cnx = mysql.connector.connect(
        host='127.0.0.1',    # Dirección del servidor MySQL (localhost)
        user='root',         # Usuario con el que nos conectamos
        password='contraseña',  # Contraseña del usuario
        use_pure=True,       # use_pure=True garantiza compatibilidad con Python 3.12, evita problemas en el Kernel
    )
    print('Conexión exitosa')  # Confirmamos que la conexión fue correcta
except mysql.connector.Error as e:  # Captura errores relacionados con MySQL
    print('Error al conectar:', e)   # Muestra el error para diagnosticar el problema


NameError: name 'mysql' is not defined

In [ ]:
# B. Creación de la base de datos

try:
    # Creamos un cursor: es el intermediario entre Python y MySQL, necesario para ejecutar consultas SQL
    mycursor = cnx.cursor()

    # Consulta SQL para crear la base de datos si no existe
    # "IF NOT EXISTS" evita errores si la base de datos ya está creada
    query_create_database = "CREATE DATABASE IF NOT EXISTS MusicStream_db"

    # Ejecutamos la consulta SQL para crear la base de datos
    mycursor.execute(query_create_database)
    print("Base de datos creada correctamente (o ya existía).")

# Captura errores relacionados con MySQL (por ejemplo, permisos insuficientes)
except mysql.connector.Error as e:  
    print("Ocurrió un error al crear la base de datos:", e)

In [ ]:
# C. Inserción de datos
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s)
"""

try:
    columnas_a_insertar = ["nombre","album","genero","año"]# indica que columnas quiero insertar del DataFrame
    df_filtrado = canciones_2016_df[columnas_a_insertar]

    df_corregido_SPOTIFY = df_filtrado.replace({np.nan: None, 'nan': None, 'NaN': None}) # corregimos valores nulos para que MySQL los entienda
    datos = df_corregido_SPOTIFY.values.tolist()

    mycursor.executemany(query_insert, datos)
    
    print(f"{mycursor.rowcount} registros insertados")
    cnx.commit() #indispensable para guardar los cambios y que se complete la petición hecha (sio da algún error antes del commit se quedan ambos programas en standby)

except Error as e:
    print("Error al insertar los datos:", e)
    cnx.rollback() # revertir petición en caso de error
    

33080 registros insertados


In [21]:
# CREACION DE TABLA INTERMEDIA QUE RELACIONA CANCIONES Y ARTISTAS 

mycursor.execute("USE MusicStream_db")

query = '''CREATE TABLE IF NOT EXISTS Cancion_artista(
	ID_Cancion INT,
    ID_Artista INT,
    PRIMARY KEY (ID_Cancion,ID_Artista),
    FOREIGN KEY (ID_Cancion) REFERENCES Canciones (ID_Cancion),
    FOREIGN KEY (ID_Artista) REFERENCES Artistas (Id_Artista)
);'''
mycursor.execute(query)


In [25]:
# INSERTAR DATOS EN TABLA Cancion_Artista

cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database="MusicStream_db"
)
mycursor = cnx.cursor(buffered=True)

try:
    # Crear lista vacía para guardar las relaciones
    relaciones = []

    # Recorremos el DataFrame fila por fila
    for _, fila in canciones_2014_df.iterrows():
        
        # Obtener ID_Cancion desde MySQL
        mycursor.execute(
            "SELECT ID_Cancion FROM Canciones WHERE Nombre = %s",
            (fila["nombre"],)
        )
        cancion = mycursor.fetchone()
        if cancion:
            id_cancion = cancion[0]
        else:
            continue  # Si no encuentra la canción, saltar esta fila

        # Obtener ID_Artista desde MySQL
        mycursor.execute(
            "SELECT ID_Artista FROM Artistas WHERE Artista = %s",
            (fila["artista"],)
        )
        artista = mycursor.fetchone()
        if artista:
            id_artista = artista[0]
        else:
            continue  # Si no encuentra el artista, saltar esta fila

        # Añadir la tupla (id_cancion, id_artista) a la lista
        relaciones.append((id_cancion, id_artista))

    # Insertar todas las relaciones de golpe usando INSERT IGNORE
    query_insert = """
    INSERT IGNORE INTO Cancion_artista (ID_Cancion, ID_Artista)
    VALUES (%s, %s)
    """
    mycursor.executemany(query_insert, relaciones)
    cnx.commit()

    print(f"{mycursor.rowcount} relaciones insertadas en Cancion_artista (IGNORE aplicado)")

except mysql.connector.Error as e:
    print("Error al insertar relaciones en Cancion_artista:", e)
    cnx.rollback()  # revertir cambios si hay error



26058 relaciones insertadas en Cancion_artista (IGNORE aplicado)


In [26]:
cnx.close()
